In [18]:
import json
import pandas as pd
import geopandas as gpd
import folium
from folium import Choropleth
from branca.colormap import linear
import ipywidgets as widgets
from IPython.display import display, HTML as ipyHTML
from IPython.display import display, IFrame
import matplotlib.pyplot as plt

In [19]:
data = pd.read_excel('dados_internet_sp.xlsx')
print(data)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [20]:
aux = data['NO_MUNICIPIO'].value_counts()
df_final = aux.reset_index()
df_final.columns = ['NO_MUNICIPIO', 'QT_ESCOLAS']
print(df_final)

                   NO_MUNICIPIO  QT_ESCOLAS
0                     São Paulo        8449
1                     Guarulhos         740
2                      Campinas         718
3         São Bernardo do Campo         511
4           São José dos Campos         482
5                      Sorocaba         478
6                Ribeirão Preto         459
7                   Santo André         458
8                        Osasco         450
9               Mogi das Cruzes         402
10                       Santos         363
11                      Jundiaí         340
12        São José do Rio Preto         327
13                   Piracicaba         294
14                       Franca         288
15                        Bauru         268
16                  São Vicente         246
17                      Taubaté         240
18                  Carapicuíba         238
19                      Limeira         220
20                      Diadema         218
21              Itaquaquecetuba 

In [21]:
df = pd.DataFrame(data)

acesso = df[df['IN_INTERNET'] == 1].groupby('NO_MUNICIPIO').size().reset_index(name='ACESSO_INTERNET')
acesso_inex = df[df['IN_INTERNET'] == 0].groupby('NO_MUNICIPIO').size().reset_index(name='ACESSO_INEX')
na = df[df['IN_INTERNET'].isnull()].groupby('NO_MUNICIPIO').size().reset_index(name='N/A')
result = pd.merge(acesso, acesso_inex, on='NO_MUNICIPIO', how='outer')
result = pd.merge(result, na, on='NO_MUNICIPIO', how='outer')
result = result.fillna(0)
print(result)

                   NO_MUNICIPIO  ACESSO_INTERNET  ACESSO_INEX     N/A
0                    Adamantina               25          1.0     2.0
1                        Adolfo                5          0.0     0.0
2                         Aguaí               27          0.0     0.0
3                        Agudos               30          0.0     4.0
4                      Alambari                7          0.0     0.0
5             Alfredo Marcondes                3          0.0     0.0
6                        Altair                4          0.0     0.0
7                   Altinópolis               13          0.0     5.0
8                   Alto Alegre                4          0.0     0.0
9                      Alumínio               15          1.0     0.0
10                  Alvinlândia                4          0.0     0.0
11                    Americana              164          1.0     7.0
12                       Amparo               57          3.0     1.0
13          Américo 

In [22]:
merged_df = pd.merge(result, df_final, on='NO_MUNICIPIO', how='inner')
merged_df.rename(columns={'NO_MUNICIPIO': 'NM_MUN'}, inplace=True)
pd.set_option('display.max_rows', None)
print(merged_df)

                         NM_MUN  ACESSO_INTERNET  ACESSO_INEX     N/A  \
0                    Adamantina               25          1.0     2.0   
1                        Adolfo                5          0.0     0.0   
2                         Aguaí               27          0.0     0.0   
3                        Agudos               30          0.0     4.0   
4                      Alambari                7          0.0     0.0   
5             Alfredo Marcondes                3          0.0     0.0   
6                        Altair                4          0.0     0.0   
7                   Altinópolis               13          0.0     5.0   
8                   Alto Alegre                4          0.0     0.0   
9                      Alumínio               15          1.0     0.0   
10                  Alvinlândia                4          0.0     0.0   
11                    Americana              164          1.0     7.0   
12                       Amparo               57   

In [23]:
merged_df['ESCOLA_ACESSO'] = merged_df['ACESSO_INTERNET'] / merged_df['QT_ESCOLAS'] * 100
merged_df['ESCOLA_INEX'] = merged_df['ACESSO_INEX'] / merged_df['QT_ESCOLAS'] * 100
merged_df['ESCOLA_NULL'] = merged_df['N/A'] / merged_df['QT_ESCOLAS']*100
print(merged_df)

                         NM_MUN  ACESSO_INTERNET  ACESSO_INEX     N/A  \
0                    Adamantina               25          1.0     2.0   
1                        Adolfo                5          0.0     0.0   
2                         Aguaí               27          0.0     0.0   
3                        Agudos               30          0.0     4.0   
4                      Alambari                7          0.0     0.0   
5             Alfredo Marcondes                3          0.0     0.0   
6                        Altair                4          0.0     0.0   
7                   Altinópolis               13          0.0     5.0   
8                   Alto Alegre                4          0.0     0.0   
9                      Alumínio               15          1.0     0.0   
10                  Alvinlândia                4          0.0     0.0   
11                    Americana              164          1.0     7.0   
12                       Amparo               57   

In [24]:
acesso_total= merged_df[merged_df['ESCOLA_ACESSO'] == 100]
acesso_totalN = acesso_total.shape[0]
print(acesso_total)
total = merged_df['QT_ESCOLAS'].sum()

                         NM_MUN  ACESSO_INTERNET  ACESSO_INEX  N/A  \
1                        Adolfo                5          0.0  0.0   
2                         Aguaí               27          0.0  0.0   
4                      Alambari                7          0.0  0.0   
5             Alfredo Marcondes                3          0.0  0.0   
6                        Altair                4          0.0  0.0   
8                   Alto Alegre                4          0.0  0.0   
10                  Alvinlândia                4          0.0  0.0   
14            Américo de Campos                5          0.0  0.0   
15                    Analândia                3          0.0  0.0   
19                      Anhumas                4          0.0  0.0   
21            Aparecida d'Oeste                3          0.0  0.0   
24                       Arandu                6          0.0  0.0   
25                       Arapeí                3          0.0  0.0   
31                  

In [25]:
print("O total de escolas com acesso total é: " + str(acesso_totalN))
print(total)

O total de escolas com acesso total é: 236
34099


In [26]:
acMIN = merged_df['ESCOLA_ACESSO'].idxmin()
linhaMIN = merged_df.loc[acMIN]
print(linhaMIN,acMIN)

NM_MUN                 Cunha
ACESSO_INTERNET           27
ACESSO_INEX             17.0
N/A                     53.0
QT_ESCOLAS                97
ESCOLA_ACESSO      27.835052
ESCOLA_INEX        17.525773
ESCOLA_NULL        54.639175
Name: 141, dtype: object 141


In [27]:
mapa_acessoint = folium.Map(location=[-23.550520, -46.633308], zoom_start=7)
colormap = linear.PiYG_09.scale(0, merged_df['ESCOLA_ACESSO'].max())
geojson_path = 'munSP.geojson'

geojson_data = json.load(open(geojson_path, encoding='utf-8'))

Choropleth(
    geo_data=geojson_data,
    data=merged_df,
    columns=['NM_MUN', 'ESCOLA_ACESSO'],
    key_on='feature.properties.NM_MUN',
    fill_color='PiYG',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Porcentagem de escolas com acesso à Internet no estado de São Paulo',
    highlight=True,
).add_to(mapa_acessoint)


colormap.add_to(mapa_acessoint)
mapa_acessoint.add_child(colormap)
mapa_acessoint
mapa_acessoint.save('escolaacessointsp.html')

In [28]:
df = pd.DataFrame(data)

ALUNOSacesso = df[df['IN_INTERNET_ALUNOS'] == 1].groupby('NO_MUNICIPIO').size().reset_index(name='ALUNOS_ACESSO_INTERNET')
ALUNOSacesso_inex = df[df['IN_INTERNET_ALUNOS'] == 0].groupby('NO_MUNICIPIO').size().reset_index(name='ALUNOS_ACESSO_INEX')
ALUNOSna = df[df['IN_INTERNET_ALUNOS'].isnull()].groupby('NO_MUNICIPIO').size().reset_index(name='ALUNOS_N/A')
resultA = pd.merge(ALUNOSacesso, ALUNOSacesso_inex, on='NO_MUNICIPIO', how='outer')
resultA = pd.merge(resultA, ALUNOSna, on='NO_MUNICIPIO', how='outer')
resultA = resultA.fillna(0)

Amerged_df = pd.merge(resultA, df_final, on='NO_MUNICIPIO', how='inner')
Amerged_df.rename(columns={'NO_MUNICIPIO': 'NM_MUN'}, inplace=True)
pd.set_option('display.max_rows', None)
print(Amerged_df)

                         NM_MUN  ALUNOS_ACESSO_INTERNET  ALUNOS_ACESSO_INEX  \
0                    Adamantina                    12.0                14.0   
1                        Adolfo                     2.0                 3.0   
2                         Aguaí                    19.0                 8.0   
3                        Agudos                    20.0                10.0   
4                      Alambari                     3.0                 4.0   
5             Alfredo Marcondes                     1.0                 2.0   
6                        Altair                     2.0                 2.0   
7                   Altinópolis                     8.0                 5.0   
8                   Alto Alegre                     2.0                 2.0   
9                      Alumínio                    12.0                 4.0   
10                  Alvinlândia                     3.0                 1.0   
11                    Americana                    8

In [29]:
Amerged_df['ESCOLA_ACESSO_ALUNOS'] = Amerged_df['ALUNOS_ACESSO_INTERNET'] / Amerged_df['QT_ESCOLAS'] * 100
Amerged_df['ESCOLA_INEX_ALUNOS'] = Amerged_df['ALUNOS_ACESSO_INEX'] / Amerged_df['QT_ESCOLAS'] * 100
Amerged_df['ESCOLA_NULL_ALUNOS'] = Amerged_df['ALUNOS_N/A'] / Amerged_df['QT_ESCOLAS']*100
print(Amerged_df)

                         NM_MUN  ALUNOS_ACESSO_INTERNET  ALUNOS_ACESSO_INEX  \
0                    Adamantina                    12.0                14.0   
1                        Adolfo                     2.0                 3.0   
2                         Aguaí                    19.0                 8.0   
3                        Agudos                    20.0                10.0   
4                      Alambari                     3.0                 4.0   
5             Alfredo Marcondes                     1.0                 2.0   
6                        Altair                     2.0                 2.0   
7                   Altinópolis                     8.0                 5.0   
8                   Alto Alegre                     2.0                 2.0   
9                      Alumínio                    12.0                 4.0   
10                  Alvinlândia                     3.0                 1.0   
11                    Americana                    8

In [30]:
Aacesso_total= Amerged_df[Amerged_df['ESCOLA_ACESSO_ALUNOS'] == 100]
Aacesso_totalN = Aacesso_total.shape[0]
print("O total de escolas com acesso total é: " + str(Aacesso_totalN))

O total de escolas com acesso total é: 32


In [31]:
mapa_acessointalunos = folium.Map(location=[-23.550520, -46.633308], zoom_start=7)
colormap = linear.RdPu_09.scale(0, Amerged_df['ESCOLA_ACESSO_ALUNOS'].max())
geojson_path = 'munSP.geojson'

geojson_data = json.load(open(geojson_path, encoding='utf-8'))

Choropleth(
    geo_data=geojson_data,
    data=Amerged_df,
    columns=['NM_MUN', 'ESCOLA_ACESSO_ALUNOS'],
    key_on='feature.properties.NM_MUN',
    fill_color='RdPu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Porcentagem de escolas com acesso à Internet para uso dos alunos no estado de São Paulo',
    highlight=True,
).add_to(mapa_acessointalunos)


colormap.add_to(mapa_acessointalunos)
mapa_acessointalunos.add_child(colormap)
mapa_acessointalunos.save('escolaacessointalunossp.html')

In [32]:
df = pd.DataFrame(data)
qlabs = df[df['IN_LABORATORIO_INFORMATICA'] == 1].groupby('NO_MUNICIPIO').size().reset_index(name='QT_LABS')
labs = pd.merge(df_final, qlabs, left_on='NO_MUNICIPIO', right_on='NO_MUNICIPIO', how='left')
#Municípios no GeoJSON que não estão no 'labs': {'Alto Alegre', 'Restinga', 'Arapeí', 'Taiúva', 'Dirce Reis', 'Guaimbê', 'Guarantã', 'Sebastianópolis do Sul'}
#São colocados valores 'NaN' em QT_LABS com 0 para os municípios sem laboratórios
labs['QT_LABS'].fillna(0, inplace=True)
labs.rename(columns={'NO_MUNICIPIO': 'NM_MUN'}, inplace=True)
labs['ESCOLA_LABS'] = labs['QT_LABS'] / labs['QT_ESCOLAS'] * 100
print(labs)

                         NM_MUN  QT_ESCOLAS  QT_LABS  ESCOLA_LABS
0                     São Paulo        8449   2541.0    30.074565
1                     Guarulhos         740    255.0    34.459459
2                      Campinas         718    280.0    38.997214
3         São Bernardo do Campo         511    188.0    36.790607
4           São José dos Campos         482    159.0    32.987552
5                      Sorocaba         478    123.0    25.732218
6                Ribeirão Preto         459    181.0    39.433551
7                   Santo André         458    214.0    46.724891
8                        Osasco         450    154.0    34.222222
9               Mogi das Cruzes         402     98.0    24.378109
10                       Santos         363    140.0    38.567493
11                      Jundiaí         340     66.0    19.411765
12        São José do Rio Preto         327    158.0    48.318043
13                   Piracicaba         294     87.0    29.591837
14        

In [33]:
labs_informaticasp = folium.Map(location=[-23.550520, -46.633308], zoom_start=7)
colormap = linear.RdBu_09.scale(0,labs['ESCOLA_LABS'].max())
geojson_path = 'munSP.geojson'

geojson_data = json.load(open(geojson_path, encoding='utf-8'))

Choropleth(
    geo_data=geojson_data,
    data=labs,
    columns=['NM_MUN', 'ESCOLA_LABS'],
    key_on='feature.properties.NM_MUN',
    fill_color='RdBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Porcentagem de escolas com laboratórios de informática',
    highlight=True,
).add_to(labs_informaticasp)

colormap.add_to(labs_informaticasp)
labs_informaticasp.add_child(colormap)
labs_informaticasp
labs_informaticasp.save('labsinfosp.html')

In [34]:
df = pd.DataFrame(data)

desktop = df[df['IN_DESKTOP_ALUNO'] == 1].groupby('NO_MUNICIPIO').size().reset_index(name='DESKTOP')
port = df[df['IN_COMP_PORTATIL_ALUNO'] == 1].groupby('NO_MUNICIPIO').size().reset_index(name='PORTATIL')
tablet = df[df['IN_TABLET_ALUNO'] == 1].groupby('NO_MUNICIPIO').size().reset_index(name='TABLET')

equip = pd.merge(desktop, port, on='NO_MUNICIPIO', how='outer')
equip = pd.merge(equip, tablet, on='NO_MUNICIPIO', how='outer')
equip = equip.fillna(0)

equip_df = pd.merge(equip, df_final, on='NO_MUNICIPIO', how='inner')
equip_df.rename(columns={'NO_MUNICIPIO': 'NM_MUN'}, inplace=True)
print(equip_df)

                         NM_MUN  DESKTOP  PORTATIL  TABLET  QT_ESCOLAS
0                    Adamantina     12.0      11.0     3.0          28
1                        Adolfo      3.0       1.0     1.0           5
2                         Aguaí     19.0      11.0    13.0          27
3                        Agudos     15.0      14.0     7.0          34
4                      Alambari      2.0       2.0     1.0           7
5             Alfredo Marcondes      2.0       2.0     1.0           3
6                        Altair      2.0       1.0     0.0           4
7                   Altinópolis      9.0       3.0     1.0          18
8                   Alto Alegre      2.0       1.0     0.0           4
9                      Alumínio     12.0       2.0     3.0          16
10                  Alvinlândia      3.0       3.0     1.0           4
11                    Americana     74.0      72.0    33.0         172
12                       Amparo     32.0      13.0     9.0          61
13    